# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,0.38,98,100,0.97,GS,1730946100
1,1,lebu,-37.6167,-73.6500,10.50,78,99,5.85,CL,1730946101
2,2,margaret river,-33.9500,115.0667,15.33,58,85,5.31,AU,1730945964
3,3,jaragua,-15.7569,-49.3344,21.16,96,100,0.94,BR,1730946104
4,4,portland,45.5234,-122.6762,11.91,74,0,2.68,US,1730945960


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_1= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles= "OSM",
    frame_width= 800,
    frame_height= 600,
    size= "humidity",
    color= 'City',
    alpha= .75
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,87,dos de mayo,-27.0228,-54.6867,21.49,99,0,0.54,AR,1730946202
128,128,mian channun,30.4400,72.3543,22.15,42,0,2.94,PK,1730946250
133,133,christchurch,-43.5333,172.6333,24.41,52,0,3.13,NZ,1730945700
231,231,saint-pierre,-21.3393,55.4781,21.82,68,0,1.54,RE,1730946374
254,254,maceio,-9.6658,-35.7353,22.69,94,0,2.57,BR,1730946401
261,261,prado,-17.3411,-39.2208,23.23,87,0,3.33,BR,1730946409
323,323,timaru,-44.4000,171.2500,21.71,63,0,1.79,NZ,1730946305
333,333,charkhi dadri,28.6000,76.2667,21.82,40,0,1.95,IN,1730946495
344,344,nabisar,25.0685,69.6419,24.36,54,0,3.45,PK,1730946507
354,354,zhangzhou,24.5133,117.6556,21.12,50,0,1.37,CN,1730946519


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""
# Display sample data
hotel_df.head(10)

C:\Users\dec21\AppData\Local\Temp\ipykernel_28164\856704128.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]= ""


,City,Country,Lat,Lng,Humidity,Hotel Name
87,dos de mayo,AR,-27.0228,-54.6867,99,
128,mian channun,PK,30.4400,72.3543,42,
133,christchurch,NZ,-43.5333,172.6333,52,
231,saint-pierre,RE,-21.3393,55.4781,68,
254,maceio,BR,-9.6658,-35.7353,94,
261,prado,BR,-17.3411,-39.2208,87,
323,timaru,NZ,-44.4000,171.2500,63,
333,charkhi dadri,IN,28.6000,76.2667,40,
344,nabisar,PK,25.0685,69.6419,54,
354,zhangzhou,CN,24.5133,117.6556,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "Categories": "accomodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat= row["Lat"]
    lon= row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dos de mayo - nearest hotel: No hotel found
mian channun - nearest hotel: No hotel found
christchurch - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
maceio - nearest hotel: No hotel found
prado - nearest hotel: No hotel found
timaru - nearest hotel: No hotel found
charkhi dadri - nearest hotel: No hotel found
nabisar - nearest hotel: No hotel found
zhangzhou - nearest hotel: No hotel found
freetown - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
ilha de mocambique - nearest hotel: No hotel found
tionk essil - nearest hotel: No hotel found
birendranagar - nearest hotel: No hotel found
numan - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
dalbandin - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
douentza - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
87,dos de mayo,AR,-27.0228,-54.6867,99,No hotel found
128,mian channun,PK,30.4400,72.3543,42,No hotel found
133,christchurch,NZ,-43.5333,172.6333,52,No hotel found
231,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
254,maceio,BR,-9.6658,-35.7353,94,No hotel found
261,prado,BR,-17.3411,-39.2208,87,No hotel found
323,timaru,NZ,-44.4000,171.2500,63,No hotel found
333,charkhi dadri,IN,28.6000,76.2667,40,No hotel found
344,nabisar,PK,25.0685,69.6419,54,No hotel found
354,zhangzhou,CN,24.5133,117.6556,50,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)